In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
import pickle
import numpy as np
from nltk.corpus import stopwords
import re
import string
import nltk

In [2]:
stop_words = stopwords.words('english')
addition = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*','’', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
stop_words.extend(addition)

In [3]:
data = pd.read_csv('Reviews.csv')

In [4]:
data.dropna(how='any',inplace=True)

In [5]:
data.drop_duplicates(inplace=True, subset=['Score','Text'])

In [6]:
idx = data[data["HelpfulnessNumerator"]>data["HelpfulnessDenominator"]].index
data.drop(index=idx, inplace=True)

In [7]:
data['Score'].value_counts()

5    250737
4     56073
1     36277
3     29770
2     20802
Name: Score, dtype: int64

In [8]:
class_1 = data.loc[data.Score==1].sample(25000)
class_2 = data.loc[data.Score==2].sample(15000)
class_3 = data.loc[data.Score==3].sample(29770)
class_4 = data.loc[data.Score==4].sample(15000)
class_5 = data.loc[data.Score==5].sample(25000)
data = pd.concat([class_1,class_2,class_3,class_4,class_5])
data.shape

(109770, 10)

In [9]:
def create_target(x):
    if x > 3:
        return 2
    elif x < 3:
        return 0
    else:
        return 1
data['target'] = data['Score'].apply(create_target)

In [10]:
data = data.sample(frac = 1)

In [11]:
file_to_read = open("glove_dictionary_100d", "rb")
glove_model = pickle.load(file_to_read)

In [12]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tweet_tokens = nltk.word_tokenize(text)
    #filtered_words = [w for w in tweet_tokens if not w in stop_words]
    #sent =" ".join(filtered_words)
    sent =" ".join(tweet_tokens)
    return sent

data['Text'] = data['Text'].apply(preprocess)

In [14]:
data.Text.apply(lambda x: len(x.split(" "))).median()

62.0

In [16]:
sentences_train = data['Text'].values
y_train = data['target'].values

y_cat = to_categorical(y_train)

In [17]:
X_t=[]
cnt=0
t_lst=[0.001 for i in range(100)]
for s_i in range(0,109770):
  my_lst=[]
  sent=sentences_train[s_i]
  word_lst=word_tokenize(sent)
  len_w_l=len(word_lst)
  for i in range(60):
    try:
      if(i<len_w_l):
        my_lst.append(glove_model[word_lst[i]])
      else:
        my_lst.append(t_lst)  
    except:
      cnt+=1
      my_lst.append(t_lst)
  X_t.append(my_lst)
print(cnt)

105554


In [18]:
X_train=np.array(X_t)
print("X_train shape = ",X_train.shape)

X_train shape =  (109770, 60, 100)


In [19]:
try:
    geeky_file = open('X_train_final', 'wb')
    pickle.dump(X_train, geeky_file)
    geeky_file.close()
  
except Exception as e:
    print(e)
    print("Something went wrong")
    
print(y_train.shape)
y_train_2=np.reshape(y_cat,(109770,1,3))

try:
    geeky_file = open('y_train_final', 'wb')
    pickle.dump(y_train_2, geeky_file)
    geeky_file.close()
  
except:
    print("Something went wrong")
    
print("y_train_2 shape = ",y_train_2.shape)

(109770,)
y_train_2 shape =  (109770, 1, 3)


In [21]:
import pickle
file_to_read = open("X_train_final", "rb")
X = pickle.load(file_to_read)
file_to_read = open("y_train_final", "rb")
y = pickle.load(file_to_read)

In [22]:
X_train = X[:76839]
X_test = X[76839:]
y_train = y[:76839]
y_test = y[76839:]

In [23]:
import tensorflow as tf
title_input = tf.keras.Input(
    shape=(60,100), name="title"
)
y1=tf.keras.layers.Conv1D(32,2,activation='relu',input_shape=(60,100))(title_input)
y2=tf.keras.layers.Conv1D(32,2,activation='relu',input_shape=(60,100))(title_input)
y3=tf.keras.layers.Conv1D(32,3,activation='relu',input_shape=(60,100))(title_input)
y4=tf.keras.layers.Conv1D(32,3,activation='relu',input_shape=(60,100))(title_input)
y5=tf.keras.layers.Conv1D(32,4,activation='relu',input_shape=(60,100))(title_input)
y6=tf.keras.layers.Conv1D(32,4,activation='relu',input_shape=(60,100))(title_input)

m1=tf.keras.layers.MaxPooling1D(pool_size=59,strides=1)(y1)
m2=tf.keras.layers.MaxPooling1D(pool_size=59,strides=1)(y2)
m3=tf.keras.layers.MaxPooling1D(pool_size=58,strides=1)(y3)
m4=tf.keras.layers.MaxPooling1D(pool_size=58,strides=1)(y4)
m5=tf.keras.layers.MaxPooling1D(pool_size=57,strides=1)(y5)
m6=tf.keras.layers.MaxPooling1D(pool_size=57,strides=1)(y6)

concanate = tf.keras.layers.concatenate([m1,m2,m3,m4,m5,m6])

den_lay1 = tf.keras.layers.Dense(256, name="MLP1")(concanate)

den_lay2 = tf.keras.layers.Dense(128, name="MLP2")(den_lay1)

den_lay3 = tf.keras.layers.Dense(64, name="MLP3")(den_lay2)

den_lay4 = tf.keras.layers.Dense(32, name="MLP4")(den_lay3)

den_lay5 = tf.keras.layers.Dense(16, name="MLP5")(den_lay4)

op_lay = tf.keras.layers.Dense(3, activation='softmax',name="priority")(den_lay5)

model = tf.keras.Model(
    inputs=[title_input],
    outputs=[op_lay],
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10)


Epoch 1/10
2402/2402 [==============================] - 40s 17ms/step - loss: 1.0163 - accuracy: 0.5865
Epoch 2/10
2402/2402 [==============================] - 38s 16ms/step - loss: 0.7798 - accuracy: 0.6492
Epoch 3/10
2402/2402 [==============================] - 37s 15ms/step - loss: 0.7336 - accuracy: 0.6741
Epoch 4/10
2402/2402 [==============================] - 40s 17ms/step - loss: 0.6868 - accuracy: 0.6992
Epoch 5/10
2402/2402 [==============================] - 43s 18ms/step - loss: 0.6482 - accuracy: 0.72000s - loss: 0.6483 - accura
Epoch 6/10
2402/2402 [==============================] - 38s 16ms/step - loss: 0.6089 - accuracy: 0.7405
Epoch 7/10
2402/2402 [==============================] - 36s 15ms/step - loss: 0.5743 - accuracy: 0.7551
Epoch 8/10
2402/2402 [==============================] - 35s 15ms/step - loss: 0.5410 - accuracy: 0.7722
Epoch 9/10
2402/2402 [==============================] - 32s 13ms/step - loss: 0.5130 - accuracy: 0.7847
Epoch 10/10
2402/2402 [===============

In [24]:
score=model.evaluate(X_test,y_test)
print(score[1])

1030/1030 [==============================] - 5s 5ms/step - loss: 0.9529 - accuracy: 0.6331
0.6331419944763184
